In [1]:
from galore_torch import GaLoreAdamW

In [2]:
from datasets import load_dataset
from torchvision import transforms
from transformers import ViTForImageClassification, AutoConfig, TrainingArguments, Trainer
import torch
from PIL import Image
import numpy as np

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, AutoConfig, TrainingArguments, Trainer
import torch

In [4]:
dataset = load_dataset("cifar10")

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),                    
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)), 
])


train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)


from datasets import Dataset
def convert_to_hf_dataset(pytorch_dataset):
    
    images = [data[0].numpy() for data in pytorch_dataset]
    labels = [data[1] for data in pytorch_dataset]
    return Dataset.from_dict({"pixel_values": images, "labels": labels})

hf_train_dataset = convert_to_hf_dataset(train_dataset)
hf_test_dataset = convert_to_hf_dataset(test_dataset)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
from transformers import ViTConfig, ViTForImageClassification
#from torchsummary import summary
import torch

config = ViTConfig(
    image_size=32,           
    patch_size=4,            
    num_channels=3,          
    hidden_size=384,         
    num_hidden_layers=7,     
    num_attention_heads=12,   
    intermediate_size=384,  
    num_labels=10           
)

In [7]:
model = ViTForImageClassification(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(4, 4), stride=(4, 4))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-6): 7 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=384, ou

In [8]:
training_args = TrainingArguments(
    output_dir="./results",           
    evaluation_strategy="epoch",           
    save_strategy="no",              
    learning_rate=5e-4,              
    per_device_train_batch_size=512,  
    per_device_eval_batch_size=512,   
    num_train_epochs=5,              
    weight_decay=0.01,               
    logging_dir="./logs",            
    logging_steps=10,                
    remove_unused_columns=True,      
    report_to="none",                

)

In [28]:
model_parameters = list(model.named_parameters())

galore_params = []
non_galore_params = []

for name, param in model_parameters:
    if ("attention" in name or "intermediate" in name) and param.ndim == 2:
        galore_params.append(param)
    else:
        non_galore_params.append(param)

param_groups = [
    {'params': non_galore_params},  
    {
        'params': galore_params,
        'rank': 64,  
        'update_proj_gap': 50,
        'scale': 0.10,
        'proj_type': 'std',  
    },
]

optimizer = GaLoreAdamW(param_groups, lr=5e-4)

/teamspace/studios/this_studio/Galore/galore_torch/adamw.py:49: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[InlineProfilerCallback()],  
)


In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.802100,1.761530,0.360500
2,1.581900,1.568274,0.419700
3,1.488600,1.487626,0.448400
4,1.377400,1.379276,0.495400
5,1.320900,1.321116,0.521700
6,1.227800,1.269860,0.536700
7,1.211600,1.237881,0.552200


Epoch 1 completed in 151.27 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 2 completed in 149.92 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 3 completed in 149.25 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 4 completed in 150.11 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 5 completed in 150.69 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 6 completed in 151.10 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB
Epoch 7 completed in 149.86 seconds
Peak GPU memory usage: 4.646 GB
Current GPU memory usage: 0.174 GB
Cached GPU memory: 5.088 GB


TrainOutput(global_step=686, training_loss=1.473583958239319, metrics={'train_runtime': 1216.3378, 'train_samples_per_second': 287.749, 'train_steps_per_second': 0.564, 'total_flos': 4.0441347072e+16, 'train_loss': 1.473583958239319, 'epoch': 7.0})

In [9]:
import torch
import time
from transformers import TrainerCallback

class InlineProfilerCallback(TrainerCallback):
    def __init__(self):
        self.epoch_start_time = None

    def on_epoch_begin(self, args, state, control, **kwargs):
        self.epoch_start_time = time.time()  
        torch.cuda.reset_peak_memory_stats()  

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch_time = time.time() - self.epoch_start_time  
        peak_memory = torch.cuda.max_memory_allocated() / 1e9  
        current_memory = torch.cuda.memory_allocated() / 1e9   
        cached_memory = torch.cuda.memory_reserved() / 1e9    

        current_epoch = int(state.epoch)
        print(f"Epoch {current_epoch} completed in {epoch_time:.2f} seconds")
        print(f"Peak GPU memory usage: {peak_memory:.3f} GB")
        print(f"Current GPU memory usage: {current_memory:.3f} GB")
        print(f"Cached GPU memory: {cached_memory:.3f} GB")

# randomized svd

In [10]:
model_parameters = list(model.named_parameters())


galore_params = []
non_galore_params = []

for name, param in model_parameters:
    if ("attention" in name or "intermediate" in name) and param.ndim == 2:
        galore_params.append(param)
    else:
        non_galore_params.append(param)

param_groups = [
    {
        'params': non_galore_params
    },
    {
        'params': galore_params,
        'rank': 64,
        'update_proj_gap': 100,
        'scale': 0.25,
        'proj_type': 'std',
        'use_randomized_svd': True  # Enable randomized SVD
    }
]

optimizer = GaLoreAdamW(param_groups, lr=5e-4)

/teamspace/studios/this_studio/Galore/galore_torch/adamw.py:49: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[InlineProfilerCallback()],  
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.779000,1.714048,0.369900
2,1.510100,1.495061,0.451000
3,1.360300,1.380858,0.498900
4,1.277600,1.293311,0.532300
5,1.219600,1.222517,0.556000
6,1.125900,1.173395,0.579800
7,1.102300,1.148384,0.593100


Epoch 1 completed in 151.40 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.918 GB
Epoch 2 completed in 148.79 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.918 GB
Epoch 3 completed in 148.07 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.918 GB
Epoch 4 completed in 148.45 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.918 GB
Epoch 5 completed in 148.76 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.920 GB
Epoch 6 completed in 147.76 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.920 GB
Epoch 7 completed in 148.43 seconds
Peak GPU memory usage: 4.538 GB
Current GPU memory usage: 0.066 GB
Cached GPU memory: 4.920 GB


TrainOutput(global_step=686, training_loss=1.3875788662245947, metrics={'train_runtime': 1206.6163, 'train_samples_per_second': 290.067, 'train_steps_per_second': 0.569, 'total_flos': 4.0441347072e+16, 'train_loss': 1.3875788662245947, 'epoch': 7.0})

# normal svd

In [18]:
model_parameters = list(model.named_parameters())


galore_params = []
non_galore_params = []

for name, param in model_parameters:
    if ("attention" in name or "intermediate" in name) and param.ndim == 2:
        galore_params.append(param)
    else:
        non_galore_params.append(param)

param_groups = [
    {
        'params': non_galore_params
    },
    {
        'params': galore_params,
        'rank': 64,
        'update_proj_gap': 100,
        'scale': 0.25,
        'proj_type': 'std',
        'use_randomized_svd': False  # Enable randomized SVD
    }
]

optimizer = GaLoreAdamW(param_groups, lr=5e-4)

/teamspace/studios/this_studio/Galore/galore_torch/adamw.py:49: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[InlineProfilerCallback()],  
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.772200,1.706230,0.365000
2,1.520700,1.527251,0.440200
3,1.367800,1.363507,0.504400
4,1.278900,1.283744,0.527500
5,1.213600,1.245465,0.545000
6,1.138000,1.201433,0.562700
7,1.113700,1.166211,0.578400


Epoch 1 completed in 149.34 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 2 completed in 149.81 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 3 completed in 149.17 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 4 completed in 149.21 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 5 completed in 149.28 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 6 completed in 149.41 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB
Epoch 7 completed in 149.23 seconds
Peak GPU memory usage: 4.605 GB
Current GPU memory usage: 0.133 GB
Cached GPU memory: 5.050 GB


TrainOutput(global_step=686, training_loss=1.3912460901299302, metrics={'train_runtime': 1210.6872, 'train_samples_per_second': 289.092, 'train_steps_per_second': 0.567, 'total_flos': 4.0441347072e+16, 'train_loss': 1.3912460901299302, 'epoch': 7.0})

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [14]:
model = ViTForImageClassification(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(4, 4), stride=(4, 4))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-6): 7 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=384, ou

In [15]:
training_args = TrainingArguments(
    output_dir="./results",           
    evaluation_strategy="epoch",           
    save_strategy="no",              
    learning_rate=5e-4,              
    per_device_train_batch_size=512,  
    per_device_eval_batch_size=512,   
    num_train_epochs=5,              
    weight_decay=0.01,               
    logging_dir="./logs",            
    logging_steps=10,                
    remove_unused_columns=True,      
    report_to="none",                

)

# rSVD

In [10]:
model_parameters = list(model.named_parameters())

galore_params = []
non_galore_params = []

for name, param in model_parameters:
    if ("attention" in name or "intermediate" in name) and param.ndim == 2:
        galore_params.append(param)
    else:
        non_galore_params.append(param)

param_groups = [
    {
        'params': non_galore_params
    },
    {
        'params': galore_params,
        'rank': 128,
        'update_proj_gap': 10,
        'scale': 0.25,
        'proj_type': 'std',
        'use_randomized_svd': True  # Enable randomized SVD
    }
]

optimizer = GaLoreAdamW(param_groups, lr=5e-4)

/teamspace/studios/this_studio/Galore/galore_torch/adamw.py:49: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[InlineProfilerCallback()],  
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.772200,1.693816,0.377400
2,1.534400,1.539204,0.431300
3,1.412800,1.417106,0.485400
4,1.326100,1.333881,0.514200
5,1.285100,1.294585,0.530800


Epoch 1 completed in 150.15 seconds
Peak GPU memory usage: 4.548 GB
Current GPU memory usage: 0.076 GB
Cached GPU memory: 4.933 GB
Epoch 2 completed in 148.51 seconds
Peak GPU memory usage: 4.548 GB
Current GPU memory usage: 0.076 GB
Cached GPU memory: 4.935 GB
Epoch 3 completed in 147.00 seconds
Peak GPU memory usage: 4.548 GB
Current GPU memory usage: 0.076 GB
Cached GPU memory: 4.935 GB
Epoch 4 completed in 147.57 seconds
Peak GPU memory usage: 4.548 GB
Current GPU memory usage: 0.076 GB
Cached GPU memory: 4.935 GB
Epoch 5 completed in 146.99 seconds
Peak GPU memory usage: 4.548 GB
Current GPU memory usage: 0.076 GB
Cached GPU memory: 4.935 GB


TrainOutput(global_step=490, training_loss=1.5264379326178104, metrics={'train_runtime': 853.6893, 'train_samples_per_second': 292.847, 'train_steps_per_second': 0.574, 'total_flos': 2.888667648e+16, 'train_loss': 1.5264379326178104, 'epoch': 5.0})

# normal

In [16]:
model_parameters = list(model.named_parameters())

galore_params = []
non_galore_params = []

for name, param in model_parameters:
    if ("attention" in name or "intermediate" in name) and param.ndim == 2:
        galore_params.append(param)
    else:
        non_galore_params.append(param)

param_groups = [
    {
        'params': non_galore_params
    },
    {
        'params': galore_params,
        'rank': 128,
        'update_proj_gap': 10,
        'scale': 0.25,
        'proj_type': 'std',
        'use_randomized_svd': False  # Enable randomized SVD
    }
]

optimizer = GaLoreAdamW(param_groups, lr=5e-4)

/teamspace/studios/this_studio/Galore/galore_torch/adamw.py:49: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[InlineProfilerCallback()],  
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.772400,1.731964,0.375100
2,1.549400,1.557702,0.425500
3,1.442300,1.430607,0.478100
4,1.339900,1.345578,0.513900
5,1.295400,1.300729,0.529000


Epoch 1 completed in 153.92 seconds
Peak GPU memory usage: 4.586 GB
Current GPU memory usage: 0.115 GB
Cached GPU memory: 5.023 GB
Epoch 2 completed in 153.60 seconds
Peak GPU memory usage: 4.586 GB
Current GPU memory usage: 0.115 GB
Cached GPU memory: 5.023 GB
Epoch 3 completed in 153.82 seconds
Peak GPU memory usage: 4.586 GB
Current GPU memory usage: 0.115 GB
Cached GPU memory: 5.023 GB
Epoch 4 completed in 156.22 seconds
Peak GPU memory usage: 4.586 GB
Current GPU memory usage: 0.115 GB
Cached GPU memory: 5.023 GB
Epoch 5 completed in 156.83 seconds
Peak GPU memory usage: 4.586 GB
Current GPU memory usage: 0.115 GB
Cached GPU memory: 5.023 GB


TrainOutput(global_step=490, training_loss=1.5406883122969648, metrics={'train_runtime': 888.7109, 'train_samples_per_second': 281.306, 'train_steps_per_second': 0.551, 'total_flos': 2.888667648e+16, 'train_loss': 1.5406883122969648, 'epoch': 5.0})